!pip install -r requirements.txt

# 0. Packages, env and clients

In [1]:
import os

from dotenv import load_dotenv
from selenium import webdriver
from bs4 import BeautifulSoup
from mistralai import Mistral

import psycopg2

load_dotenv()

True

In [2]:
# Environment variables
EMAIL_MONDE = os.getenv("EMAIL_MONDE")
PASSWORD_MONDE = os.getenv("PASSWORD_MONDE")
USER_PGSQL = os.getenv("USER_PGSQL")
PASSWORD_PGSQL = os.getenv("PASSWORD_PGSQL")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
MODEL = "mistral-large-latest"

# URLs
URL_ARTICLE = "https://www.lemonde.fr/idees/article/2025/09/12/comment-la-politique-de-l-offre-detourne-l-argent-public-au-profit-des-plus-riches_6640595_3232.html"
URL_ARTICLE = "https://www.lemonde.fr/idees/article/2025/09/14/patrick-artus-aux-etats-unis-le-capitalisme-de-monopole-provoque-sous-investissement-et-concentration-de-la-richesse_6641078_3232.html"
URL_ARTICLE = "https://www.lemonde.fr/politique/article/2025/09/14/a-bordeaux-marine-le-pen-predit-la-chute-de-lecornu-et-promet-des-lendemains-qui-chantent_6641191_823448.html"
URL_COMMENTAIRES = URL_ARTICLE + "?contributions&page="

In [3]:
connection = psycopg2.connect(
            host="localhost",        # ou "127.0.0.1"
            port="5432",            # port par défaut PostgreSQL
            database="LeMondeComments",  # remplacez par le nom de votre base
            user=USER_PGSQL,   # remplacez par votre nom d'utilisateur
            password=PASSWORD_PGSQL # remplacez par votre mot de passe
        )

In [4]:
client = Mistral(api_key=MISTRAL_API_KEY)

# 1. Data acquisition and storage

In [5]:
# Create a new instance of the Chrome driver
driver = webdriver.Firefox()

# Open Google
#driver.get("https://www.lemonde.fr/")
driver.get("https://secure.lemonde.fr/sfuser/connexion")

input_element = driver.find_element("id", "email")
input_element.send_keys(EMAIL_MONDE)

password_input = driver.find_element("id", "password")
password_input.send_keys(PASSWORD_MONDE)

validate_button = driver.find_element("id", "submit-button")
validate_button.click()

all_comments = []

next_page = True
page = 0

while next_page:

    page += 1
    print(f"Loading page {page}...")
    driver.get(URL_COMMENTAIRES + str(page))
    #time.sleep(2)  # Wait for 2 seconds to allow the page to load

    html = driver.page_source

    soup = BeautifulSoup(html)
    #soup = BeautifulSoup(html, 'html.parser')

    # Find all elements with class="comment_content"
    comment_contents = soup.find_all(class_="comment__content")

    if len(comment_contents) == 0:
        next_page = False
        print("No more comments found.")
        break

    comment_list = [comment.text for comment in comment_contents]

    all_comments.extend(comment_list)

# Close the browser
driver.quit()

Loading page 1...
Loading page 2...
Loading page 3...
Loading page 4...
Loading page 5...
No more comments found.


In [6]:
print(f"Il y a en tout {len(all_comments)} commentaires sur cet article.")

Il y a en tout 89 commentaires sur cet article.


In [7]:
comment_txt = "\n".join(all_comments)

In [8]:
cursor = connection.cursor()
insert_query = "INSERT INTO comments (title, url, txt) VALUES (%s, %s, %s);"
values = ('Coucou', URL_ARTICLE, comment_txt)
cursor.execute(insert_query, values)
connection.commit()
cursor.close()

# 2. Analyse des commentaires

In [9]:
PROMPT_BOT = f"""Tu es un expert détection d'ingérence par des bot russes.
Parmi tous les commentaires suivantes, séparés par des retours à la ligne, indique celui qui est le plus à même d'être issu d'un bot russe :{comment_txt}"""

In [10]:
PROMPT_ANALYSE = f"""Tu es un expert en analyse de texte. 
Tu vas analyser les commentaires suivants issus d'un article du journal Le Monde. Chaque commentaire est séparé par un saut de ligne.
Donne moi la liste des thèmes abordés dans ces commentaires, ainsi que le nombre d'occurrences pour chaque thème.
Voici les commentaires à analyser :{comment_txt}"""

In [11]:
PROMPT_ANALYSE = f"""Tu es un expert en analyse de texte. 
Tu vas analyser les commentaires suivants issus d'un article du journal Le Monde. Chaque commentaire est séparé par un saut de ligne.
Voici les commentaires à analyser :{comment_txt}"""

In [12]:
chat_response = client.chat.complete(
    model= MODEL,
    messages = [
        {
            "role": "user",
            "content": PROMPT_ANALYSE,
        },
    ]
)
print(chat_response.choices[0].message.content)

Voici une analyse structurée des commentaires issus de l’article du *Monde*, organisée selon les **thèmes récurrents**, les **stratégies rhétoriques**, les **biais cognitifs**, et les **clivages politiques** qu’ils révèlent. Cette analyse met en lumière les dynamiques discursives autour du Rassemblement National (RN), de La France Insoumise (LFI), et du contexte politique français actuel.

---

### **1. Thèmes dominants**
#### **A. La peur de l’extrême droite et les références historiques**
- **Comparaisons avec les régimes autoritaires** :
  - *"Le Pen et Mélenchon main dans la main, comme Hitler et Staline"* (pacte germano-soviétique de 1939).
  - *"La couleur [de la révolution] est brune"* (référence au nazisme).
  - *"Espagne 1936, Chili 1974"* (guerre civile et coup d’État de Pinochet).
  - *"Maréchal nous voilà"* (chant collaborationniste de Vichy).
  **Fonction** : Diabolisation du RN via des **amalgames historiques**, avec une **rhétorique de l’urgence** ("l’histoire ne leur a 